In [2]:
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import feature_handler as handler

## Import data
- name : stock list
- date : date we collected
- clear : stock data we crawl dowm

In [6]:
path = "./Data/" 
allData = pd.read_csv(path+'D2_data.csv')


In [38]:
Stock_list = pd.Series(allData.STOCK_ID, dtype="category").cat.categories.tolist()
date = pd.Series(allData.Date, dtype="category").cat.categories.tolist()
Stock = allData.drop_duplicates()


columns = ['Open', 'High', 'Low', 'Close']
for c in columns:
    Stock = Stock[ Stock[c] != '#NAME?' ]
    Stock[c] = Stock[c].astype(float)
    Stock[c] -= 1.1 * min(Stock[c])
    
Stock = Stock[~Stock.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index(drop=True)

In [39]:
features =[1,5, 10, 20, 60, 120]
labels = ['Close','Close', 'Close','Close', 'Close', 'Close']

for l in range(len(labels)):  
    short_label = 'MA' + str(features[l])
    Stock[short_label] = handler.MA_index(Stock,features[l],labels[l])  

short = [1,1,5,5,5,5,10,10,20]
long = [5,60,10,20,60,120,20,60,60]
for l in range(len(short)):
    handler.MA_cross(Stock , short[l] , long[l], 'noTrend')

-1
-1
-1
-1
-1
-1
-1
-1
-1


In [40]:
day_list=[6,9,12]
for i in day_list:
    handler.MACD(Stock, i , 'noTrend')
    handler.KD_indicator(Stock, i, 'noTrend')
Stock = Stock[~Stock.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index(drop=True)

In [41]:
Stock.to_csv(path+'D2_model.csv', index=False)


In [42]:
Stock

,STOCK_ID,Date,Open,High,Low,Close,MA1,MA5,MA10,MA20,...,noTrendcross_6,noTrendover_3days_6,noTrendMACD_cross_9,noTrendoverBuyOrSold(80/20)_9,noTrendcross_9,noTrendover_3days_9,noTrendMACD_cross_12,noTrendoverBuyOrSold(80/20)_12,noTrendcross_12,noTrendover_3days_12
0,2317,19930705,2.997276,2.481707,1.655375,0.923848,0.923848,2.075983,1.863750,1.935607,...,-1,0,0,0,-1,0,0,0,-1,0
1,2317,19930706,1.047873,2.102953,1.417436,3.504984,3.504984,2.478462,2.089011,2.058362,...,1,0,0,0,1,0,0,0,1,0
2,2317,19930707,3.435044,2.268261,3.550916,1.137823,1.137823,2.429352,1.886020,2.007108,...,-1,0,0,0,-1,0,0,0,-1,0
3,2317,19930708,1.532346,0.999901,2.579427,2.280625,2.280625,2.172103,1.949576,2.041054,...,0,0,0,0,0,0,0,0,1,0
4,2317,19930709,1.858309,0.581851,1.563290,2.138767,2.138767,1.997209,2.039563,2.097421,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117393,6415,20220211,0.597005,0.490660,0.530530,0.317759,0.317759,2.122296,2.004986,2.071978,...,-1,0,-1,0,-1,0,-1,0,-1,0
117394,6415,20220214,0.563659,1.053305,0.466405,1.407881,1.407881,1.780850,1.948934,2.116726,...,0,0,0,0,0,0,0,0,0,0
117395,6415,20220215,2.363992,2.607997,2.774379,1.733663,1.733663,1.682738,2.072175,2.089197,...,0,0,0,0,0,0,0,0,0,0
117396,6415,20220216,2.703597,1.970412,2.321521,2.383746,2.383746,1.763110,2.008929,2.108944,...,1,0,0,0,1,0,0,0,1,0


In [30]:
noT_Stock.K

0          169.140533
1           85.419857
2          162.335186
3          150.676166
4          150.913256
             ...     
116954    1141.256108
116955     581.954479
116956     577.379951
116957     465.073964
116958     301.992682
Name: K, Length: 116959, dtype: object

In [8]:
noT_Stock = Stock.copy()

In [29]:
import math
from scipy.stats import ks_2samp
index = 7

TAIEX_df = noT_Stock[noT_Stock.STOCK_ID == Stock_list[index]].reset_index(drop=True)[10:].reset_index(drop=True)
ks_score = []
statistic = []
for c in noT_Stock.columns :
    temp = ks_2samp(noT_Stock[c][:math.ceil(len(noT_Stock)/2)], noT_Stock[c][math.ceil(len(noT_Stock)/2):])
    ks_score.append(temp[1])
    statistic.append(temp[0])

score_df = pd.DataFrame()
score_df['label'] = TAIEX_df.columns
score_df['ks'] = ks_score
score_df['sta'] = statistic
score_df.sort_values(by=['ks'],ascending=False)

,label,ks,sta
28,noTrendcross_6,9.999960e-01,0.001695
16,noTrendMA_cross_5_60,9.716495e-01,0.002842
14,noTrendMA_cross_5_10,9.716452e-01,0.002842
18,noTrendMA_cross_10_20,9.328561e-01,0.003150
15,noTrendMA_cross_5_20,9.300818e-01,0.003167
34,noTrendMACD_cross_12,8.472126e-01,0.003577
24,noTrendMACD_cross_6,8.472126e-01,0.003577
30,noTrendMACD_cross_9,8.472126e-01,0.003577
17,noTrendMA_cross_5_120,7.631391e-01,0.003903
32,noTrendcross_9,6.371798e-01,0.004346


In [9]:
path = "./Data/" 
allData = pd.read_csv(path+'Stock.csv')


In [10]:
Stock_list = pd.Series(allData.STOCK_ID, dtype="category").cat.categories.tolist()
date = pd.Series(allData.Date, dtype="category").cat.categories.tolist()
allData=allData.drop(['Dividends','Stock Splits','Adj Close'],axis=1)
Stock = allData.drop_duplicates()
Stock = Stock[~Stock.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index(drop=True)

In [11]:
features =[1,5, 10, 20, 60, 120]
labels = ['Close','Close', 'Close','Close', 'Close', 'Close']

for l in range(len(labels)):  
    short_label = 'MA' + str(features[l])
    Stock[short_label] = handler.MA_index(Stock,features[l],labels[l])  

short = [1,1,5,5,5,5,10,10,20]
long = [5,60,10,20,60,120,20,60,60]
for l in range(len(short)):
    handler.MA_cross(Stock , short[l] , long[l])

day_list=[6,9,12]
for i in day_list:
    handler.MACD(Stock, i)
    handler.KD_indicator(Stock, i)

-1
-1
-1
-1
-1
-1
-1
-1
-1


In [26]:
Stock_list

[1216,
 1301,
 1303,
 1326,
 2002,
 2303,
 2308,
 2317,
 2330,
 2379,
 2412,
 2454,
 2603,
 2881,
 2882,
 2884,
 2885,
 2886,
 2891,
 3034,
 3711,
 5871,
 6415]

In [27]:
index = 7
Stock = Stock[~Stock.isin([np.nan, np.inf, -np.inf]).any(1)].reset_index(drop=True)
TAIEX_df = Stock[Stock.STOCK_ID == Stock_list[index]].reset_index(drop=True)[10:].reset_index(drop=True)
ks_score = []
statistic = []
for c in Stock.columns :
    temp = ks_2samp(Stock[c][:math.ceil(len(TAIEX_df)/2)], Stock[c][math.ceil(len(TAIEX_df)/2):])
    ks_score.append(temp[1])
    statistic.append(temp[0])

score_df = pd.DataFrame()
score_df['label'] = TAIEX_df.columns
score_df['ks'] = ks_score
score_df['sta'] = statistic
score_df.sort_values(by=['ks'],ascending=False)

,label,ks,sta
29,cross_6,1.000000e+00,0.001123
25,MACD_cross_6,1.000000e+00,0.001525
37,cross_12,1.000000e+00,0.001611
35,MACD_cross_12,1.000000e+00,0.001525
13,MA_cross_1_5,1.000000e+00,0.001136
31,MACD_cross_9,1.000000e+00,0.001525
21,MA_cross_20_60,1.000000e+00,0.001814
33,cross_9,1.000000e+00,0.003197
19,MA_cross_10_20,1.000000e+00,0.003200
20,MA_cross_10_60,1.000000e+00,0.004181


In [31]:
Stock.K

0           8.81079
1          7.584263
2          11.72321
3         11.625533
4          21.33218
            ...    
117079    43.826684
117080    34.290253
117081    27.932633
117082    25.143494
117083    19.186572
Name: K, Length: 117084, dtype: object

In [28]:
Stock.to_csv(path+'model.csv', index=False)

In [23]:
len(noT_Stock)

116959

In [ ]:
# fuse

In [84]:
columns = ['開盤價','最高價','最低價','收盤價','成交股數','Dividends','Stock Splits','證券代號','Adj Close','date']
allData.columns = columns
allData.sort_values(by=['證券代號','date'] , ascending=True, axis =0 , inplace=True)
allData=allData.drop(['Dividends','Stock Splits','Adj Close'],axis=1)
Stock = allData.drop_duplicates()
Stock = Stock[~Stock.isin([np.nan, np.inf, -np.inf]).any(1)]
Stock = Stock.reset_index(drop=True)


## Build all or Update
- update flag for solving time rebuild all
- Stock_PRE : previous data for model training

In [13]:
update = False
#update = True
if update :
    Stock_PRE = pd.read_csv('model.csv')
    finall_date = max(Stock_PRE.date)
    Stock_PRE

## building feature for model

In [86]:
list_1 = []
list_2 = []
for i in Stock_list :
    s = Stock[Stock['證券代號'] == i].reset_index(drop=True)
    handler.前高(s,list_1)
    handler.波段前高(s,list_2)
Stock['年前高'] = list_1
Stock['季前高'] = list_2

In [87]:
if update :
    start_date = date[date[date.values == finall_date ].index + 80].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [88]:
handler.漲跌(Stock)

In [89]:
handler.紅棒(Stock ,"實紅棒","紅棒天線")

In [90]:
handler.黑棒(Stock , "實黑棒","黑棒天線")

In [91]:
handler.避雷針(Stock , "上避雷針","下避雷針")

In [92]:
handler.連續(Stock , "連漲跌")

In [93]:
handler.季均漲跌天數(Stock , 60)

In [94]:
handler.MACD(Stock)

In [95]:
Stock

,開盤價,最高價,最低價,收盤價,成交股數,證券代號,date,年前高,季前高,漲跌,...,實黑棒,黑棒天線,上避雷針,下避雷針,連漲跌,均漲天數,均跌天數,MACD,MACD_signal,MACD_histogram
0,6.120794,6.385764,6.120794,6.385764,48116125.0,1216,20000104,0.0,0.0,0.00,...,0.000000,0.000000,1.451084e-14,-4.329009,0,0,0,NaN,NaN,NaN
1,6.409853,6.816942,6.361676,6.771174,74735686.0,1216,20000105,0.0,0.0,6.03,...,0.000000,0.000000,7.140196e-01,-6.388578,1,0,0,NaN,NaN,NaN
2,6.841030,6.913294,6.626645,6.650733,39285404.0,1216,20000106,0.0,0.0,-1.78,...,0.931071,0.068929,3.838028e+00,-0.352112,0,0,0,NaN,NaN,NaN
3,6.650734,6.650734,6.313500,6.458029,19623075.0,1216,20000107,0.0,0.0,-2.90,...,0.588891,0.411109,2.897499e+00,-2.173124,-1,0,0,NaN,NaN,NaN
4,6.506207,6.674824,6.458031,6.554383,24277617.0,1216,20000110,0.0,0.0,1.49,...,0.000000,0.000000,1.851165e+00,-1.480932,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117198,4765.000000,4955.000000,4745.000000,4920.000000,283024.0,6415,20220105,5490.0,5490.0,2.50,...,0.000000,0.000000,7.345226e-01,-3.672613,0,2,-2,101.515870,79.853326,21.662544
117199,4665.000000,4720.000000,4430.000000,4430.000000,494029.0,6415,20220106,5490.0,5490.0,-9.96,...,1.000043,-0.000043,6.216506e+00,0.000000,-1,2,-2,57.799369,75.442535,-17.643166
117200,4340.000000,4550.000000,4340.000000,4380.000000,402143.0,6415,20220107,5490.0,5490.0,-1.13,...,4000.000000,-3999.000000,3.917051e+00,-0.921659,-2,2,-2,18.901262,64.134280,-45.233018
117201,4045.000000,4285.000000,4030.000000,4220.000000,475206.0,6415,20220110,5490.0,5490.0,-3.65,...,0.000000,0.000000,1.606922e+00,-4.697157,-2,2,-2,-24.553395,46.396745,-70.950140


In [96]:
feature = 'MA60'
label = '收盤價'

day = int(''.join([x for x in feature if x.isdigit()]))    
Stock[feature] = handler.MA_index(Stock,day,label)  

In [97]:
Stock

,開盤價,最高價,最低價,收盤價,成交股數,證券代號,date,年前高,季前高,漲跌,...,黑棒天線,上避雷針,下避雷針,連漲跌,均漲天數,均跌天數,MACD,MACD_signal,MACD_histogram,MA60
0,6.120794,6.385764,6.120794,6.385764,48116125.0,1216,20000104,0.0,0.0,0.00,...,0.000000,1.451084e-14,-4.329009,0,0,0,NaN,NaN,NaN,NaN
1,6.409853,6.816942,6.361676,6.771174,74735686.0,1216,20000105,0.0,0.0,6.03,...,0.000000,7.140196e-01,-6.388578,1,0,0,NaN,NaN,NaN,NaN
2,6.841030,6.913294,6.626645,6.650733,39285404.0,1216,20000106,0.0,0.0,-1.78,...,0.068929,3.838028e+00,-0.352112,0,0,0,NaN,NaN,NaN,NaN
3,6.650734,6.650734,6.313500,6.458029,19623075.0,1216,20000107,0.0,0.0,-2.90,...,0.411109,2.897499e+00,-2.173124,-1,0,0,NaN,NaN,NaN,NaN
4,6.506207,6.674824,6.458031,6.554383,24277617.0,1216,20000110,0.0,0.0,1.49,...,0.000000,1.851165e+00,-1.480932,0,0,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117198,4765.000000,4955.000000,4745.000000,4920.000000,283024.0,6415,20220105,5490.0,5490.0,2.50,...,0.000000,7.345226e-01,-3.672613,0,2,-2,101.515870,79.853326,21.662544,4646.583333
117199,4665.000000,4720.000000,4430.000000,4430.000000,494029.0,6415,20220106,5490.0,5490.0,-9.96,...,-0.000043,6.216506e+00,0.000000,-1,2,-2,57.799369,75.442535,-17.643166,4655.833333
117200,4340.000000,4550.000000,4340.000000,4380.000000,402143.0,6415,20220107,5490.0,5490.0,-1.13,...,-3999.000000,3.917051e+00,-0.921659,-2,2,-2,18.901262,64.134280,-45.233018,4662.916667
117201,4045.000000,4285.000000,4030.000000,4220.000000,475206.0,6415,20220110,5490.0,5490.0,-3.65,...,0.000000,1.606922e+00,-4.697157,-2,2,-2,-24.553395,46.396745,-70.950140,4665.166667


In [98]:
if update :
    start_date = date[date[date.values == finall_date ].index + 25].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [99]:
Stock['RSI']=handler.RSI(Stock.收盤價,14)

In [100]:
# adding MA feature
features =['MA5', 'MA10', 'MA20', 'buy5', 'buy10']
labels = ['收盤價', '收盤價','收盤價', '成交股數', '成交股數'] 
for l in range(len(labels)):
    day = int(''.join([x for x in features[l] if x.isdigit()]))    
    Stock[features[l]] = handler.MA_index(Stock,day,labels[l])  

In [101]:
handler.day_Volatility(Stock, '日振幅')

In [102]:
handler.Implied_Volatility(Stock,'波動率',20)

In [103]:
Stock

,開盤價,最高價,最低價,收盤價,成交股數,證券代號,date,年前高,季前高,漲跌,...,MACD_histogram,MA60,RSI,MA5,MA10,MA20,buy5,buy10,日振幅,波動率
0,6.120794,6.385764,6.120794,6.385764,48116125.0,1216,20000104,0.0,0.0,0.00,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,4.328302,NaN
1,6.409853,6.816942,6.361676,6.771174,74735686.0,1216,20000105,0.0,0.0,6.03,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,7.155261,NaN
2,6.841030,6.913294,6.626645,6.650733,39285404.0,1216,20000106,0.0,0.0,-1.78,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,4.325047,NaN
3,6.650734,6.650734,6.313500,6.458029,19623075.0,1216,20000107,0.0,0.0,-2.90,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,5.340623,NaN
4,6.506207,6.674824,6.458031,6.554383,24277617.0,1216,20000110,0.0,0.0,1.49,...,NaN,NaN,0.000000,6.564017,NaN,NaN,41207581.4,NaN,3.356434,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117198,4765.000000,4955.000000,4745.000000,4920.000000,283024.0,6415,20220105,5490.0,5490.0,2.50,...,21.662544,4646.583333,54.573626,4988.000000,4948.5,4726.75,229557.6,216792.0,4.425710,3.908225
117199,4665.000000,4720.000000,4430.000000,4430.000000,494029.0,6415,20220106,5490.0,5490.0,-9.96,...,-17.643166,4655.833333,41.548660,4852.000000,4905.0,4729.00,305734.4,233429.4,6.546274,4.061322
117200,4340.000000,4550.000000,4340.000000,4380.000000,402143.0,6415,20220107,5490.0,5490.0,-1.13,...,-45.233018,4662.916667,40.486804,4723.000000,4863.0,4723.50,347719.2,255532.8,4.838709,4.183940
117201,4045.000000,4285.000000,4030.000000,4220.000000,475206.0,6415,20220110,5490.0,5490.0,-3.65,...,-70.950140,4665.166667,37.209634,4550.000000,4805.0,4716.00,405339.2,286731.2,6.327542,4.368738


In [104]:
if update :
    start_date = date[date[date.values == finall_date ].index + 10].squeeze()
    Stock = Stock[Stock.date > start_date].reset_index(drop=True)
    Stock

In [105]:
handler.MA交叉型態(Stock)

-1


In [106]:
handler.KD(Stock)

In [107]:
handler.KD交叉型態(Stock)

In [108]:
handler.KD交叉型態2(Stock)

In [109]:
feature = ['MA5 slope','MA10 slope','MA60 slope','buy5 slope','buy10 slope','MA5 acc']
label = ['MA5','MA10','MA60','buy5','buy10','MA5 slope']
for i in range(len(label)) :
    handler.Slope(Stock , feature[i] , label[i])

In [110]:
handler.RSI鈍化(Stock)

In [111]:
feature = ['D5','D60','D_upper_day']
label1 = ['收盤價','收盤價','收盤價']
label2 = ['MA5','MA60','最高價']
for i in range(len(feature)):
    handler.distance(Stock , feature[i] , label1[i] ,label2[i])

In [112]:
feature = ['UD5','UD60','UD前高壓力','UD波段壓立']
label1 = ['收盤價','收盤價','收盤價','收盤價']
label2 = ['MA5','MA60','年前高','季前高']
for i in range(len(feature)):
    handler.Updown_state(Stock , feature[i] , label1[i] ,label2[i])

In [113]:
handler.均線糾結(Stock)

In [114]:
handler.三陽開泰 (Stock)

In [115]:
handler.突破前高(Stock)

In [116]:
handler.突破整理(Stock)

max() arg is an empty sequence 0
max() arg is an empty sequence 1
max() arg is an empty sequence 2
max() arg is an empty sequence 3
max() arg is an empty sequence 4
max() arg is an empty sequence 5
max() arg is an empty sequence 6
max() arg is an empty sequence 7
max() arg is an empty sequence 8
max() arg is an empty sequence 9


In [117]:
handler.超漲跌(Stock)

In [118]:
days = [5,3,1]
uppers = [0.1 , 0.05 ,0.03 ]
for i in range(len(days)):
    handler.shift_price(Stock, days[i] , uppers[i])

In [42]:
if update :
    Stock_new = Stock[Stock.date > finall_date].reset_index(drop = True)
    Stock_new
    Stock = Stock_PRE.append(Stock_new)
    Stock.sort_values(by=['證券代號','date'] , ascending=True, axis =0 , inplace=True)
    Stock = Stock.reset_index(drop=True)

In [46]:
Stock_index = pd.read_csv('Market_index.csv')
Stock_index.sort_values(by=['Index','date'] , ascending=True, axis =0 , inplace=True)
index_set = ['發行量加權股價指數' , '未含電子指數' , '電子類兩倍槓桿指數' ]
Stock_index = Stock_index.reset_index(drop=True)


features =['MA5', 'MA20' ]
labels = ['收盤', '收盤'] 
for l in range(len(labels)):
    test_ = []
    day = int(''.join([x for x in features[l] if x.isdigit()]))
        
    Stock_index[features[l]] = handler.MA_index(Stock_index,day,labels[l])

features =['MA5 slop', 'MA20 slop' ]
labels = ['MA5', 'MA20'] 
for l in range(len(labels)):
    handler.Slope (Stock_index , features[l] , labels[l] )

features =['乖離5', '乖離20' ]
label1 = ['收盤','收盤']
label2 = ['MA5','MA20']
for i in range(len(labels)):
    handler.distance (Stock_index , features[i] , label1[i] ,label2[i] )
    
Stock_index

,Index,收盤,漲跌,漲跌%,date,MA5,MA20,MA5 slop,MA20 slop,乖離5,乖離20
0,未含電子指數,15054.77,65.44,0.44,20190429,NaN,NaN,NaN,NaN,NaN,NaN
1,未含電子指數,15107.62,52.85,0.35,20190430,NaN,NaN,NaN,NaN,NaN,NaN
2,未含電子指數,15197.27,89.65,0.59,20190502,NaN,NaN,NaN,NaN,NaN,NaN
3,未含電子指數,15194.64,2.63,-0.02,20190503,NaN,NaN,NaN,NaN,NaN,NaN
4,未含電子指數,15004.13,190.51,-1.25,20190506,15111.686,NaN,NaN,NaN,-0.01,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2012,電子類兩倍槓桿指數,41886.84,445.67,-1.05,20211206,41909.746,41731.6925,1.007,1.002,-0.00,0.00
2013,電子類兩倍槓桿指數,42177.11,290.27,0.69,20211207,42162.036,41784.2410,1.006,1.001,0.00,0.01
2014,電子類兩倍槓桿指數,42182.06,4.95,0.01,20211208,42242.770,41826.8090,1.002,1.001,-0.00,0.01
2015,電子類兩倍槓桿指數,42590.27,408.21,0.97,20211209,42233.758,41918.6830,1.000,1.002,0.01,0.02


In [47]:
Market = Stock_index[(Stock_index.Index =='發行量加權股價指數' )]
Market

,Index,收盤,漲跌,漲跌%,date,MA5,MA20,MA5 slop,MA20 slop,乖離5,乖離20
642,發行量加權股價指數,10145.28,80.50,0.80,20190218,17993.780,19153.6320,0.904,0.976,-0.77,-0.89
643,發行量加權股價指數,10152.26,6.98,0.07,20190219,16051.006,18680.4945,0.892,0.975,-0.58,-0.84
644,發行量加權股價指數,10272.46,120.20,1.18,20190220,14111.752,18208.2005,0.879,0.975,-0.37,-0.77
645,發行量加權股價指數,10319.53,47.07,0.46,20190221,12165.282,17733.7580,0.862,0.974,-0.18,-0.72
646,發行量加權股價指數,10322.92,3.39,0.03,20190222,10242.490,17269.6395,0.842,0.974,0.01,-0.67
...,...,...,...,...,...,...,...,...,...,...,...
1324,發行量加權股價指數,17688.21,8.93,-0.05,20211206,17624.796,17620.5405,1.004,1.001,0.00,0.00
1325,發行量加權股價指數,17796.92,108.71,0.61,20211207,17698.628,17633.3185,1.004,1.001,0.01,0.01
1326,發行量加權股價指數,17832.42,35.50,0.20,20211208,17747.914,17646.9570,1.003,1.001,0.00,0.01
1327,發行量加權股價指數,17914.12,81.70,0.46,20211209,17785.762,17670.0370,1.002,1.001,0.01,0.01


In [48]:
date

0      20211210
1      20211209
2      20211208
3      20211207
4      20211206
         ...   
684    20190221
685    20190220
686    20190219
687    20190218
688    20190215
Name: date, Length: 689, dtype: int64

In [119]:
Stock.columns

Index(['開盤價', '最高價', '最低價', '收盤價', '成交股數', '證券代號', 'date', '年前高', '季前高', '漲跌',
       '實紅棒', '紅棒天線', '實黑棒', '黑棒天線', '上避雷針', '下避雷針', '連漲跌', '均漲天數', '均跌天數',
       'MACD', 'MACD_signal', 'MACD_histogram', 'MA60', 'RSI', 'MA5', 'MA10',
       'MA20', 'buy5', 'buy10', '日振幅', '波動率', 'MA交叉型態', 'K', 'D', 'KD交叉型態',
       'KD交叉型態2', 'MA5 slope', 'MA10 slope', 'MA60 slope', 'buy5 slope',
       'buy10 slope', 'MA5 acc', 'RSI5鈍化', 'D5', 'D60', 'D_upper_day', 'UD5',
       'UD60', 'UD前高壓力', 'UD波段壓立', '均線糾結1', '均線糾結2', '三陽開泰', '突破前高', '超漲跌',
       'pre5p0.1', 'pre5d0.1', 'pre3p0.05', 'pre3d0.05', 'pre1p0.03',
       'pre1d0.03'],
      dtype='object')

list_ma5 = []
list_ma20 = []
list_market = []
list_5D = []
list_20D = []
Market = Stock_index[(Stock_index.Index =='發行量加權股價指數' )]
for i in range(len(Stock)):
    day = Stock.date[i].squeeze()
    market = Market[(Market.date == int(day))]
    list_ma5.append(market['MA5 slop'].squeeze())
    list_ma20.append(market['MA20 slop'].squeeze())
    list_market.append(market.漲跌.squeeze())
    list_5D.append(market.乖離5.squeeze())
    list_20D.append(market.乖離20.squeeze())
    

Stock['大盤漲跌'] = list_market
Stock['大盤5MA'] = list_ma5
Stock['大盤20MA'] = list_ma20
Stock['大盤5乖離'] = list_5D
Stock['大盤20乖離'] = list_20D

In [121]:
Stock.to_csv(path+'model.csv', index=False)